## 1. 모듈 및 데이터셋 불러오기

In [1]:
import os 
import cv2
import torch
import torchvision
import PIL

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from glob import glob
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from ipywidgets import interact
from torchvision.transforms.functional import to_pil_image

# from save import save_model

In [2]:
data_dir = "./traincrop/"
images_paths = glob(data_dir + "*.jpg")
masks_paths = glob(data_dir + "*.png")

images_paths = sorted([str(p) for p in images_paths])
masks_paths = sorted([str(p) for p in masks_paths])

df = pd.DataFrame({'images': images_paths, 'masks': masks_paths})
df.head(5)

,images,masks
0,./traincrop\20160222_081011_1281_721.jpg,./traincrop\20160222_081011_1281_721.png
1,./traincrop\20160222_081011_1921_721.jpg,./traincrop\20160222_081011_1921_721.png
2,./traincrop\20160222_081011_1_361.jpg,./traincrop\20160222_081011_1_361.png
3,./traincrop\20160222_081011_1_721.jpg,./traincrop\20160222_081011_1_721.png
4,./traincrop\20160222_081011_641_361.jpg,./traincrop\20160222_081011_641_361.png


In [3]:
train, valid = train_test_split(df, test_size=0.1, shuffle=True, random_state=42)

print(f"Train size: {len(train)}, Validation size: {len(valid)}")

Train size: 1656, Validation size: 184


In [10]:
@interact(index=(0, len(df)-1))
def show_images(index=0):
    image = cv2.imread(df.iloc[index].images)
    mask = cv2.imread(df.iloc[index].masks)

    plt.figure(figsize=(12,10))
    plt.subplot(121)
    plt.title("image")
    plt.imshow(image)
    plt.subplot(122)
    plt.title("mask")
    plt.imshow(mask)
    plt.tight_layout()

interactive(children=(IntSlider(value=0, description='index', max=1839), Output()), _dom_classes=('widget-inte…

## 2. 데이터셋 구축

### Albumentations

In [4]:
import random
import albumentations as A

from albumentations.pytorch.transforms import ToTensorV2

In [38]:
train_transform = [
    A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=2, p=1),
    # A.RandomShadow(num_shadows_lower=1, num_shadows_upper=1, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=1),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
    A.CLAHE(clip_limit=2, tile_grid_size=(8, 8), p=1),
    # A.RGBShift(r_shift_limit=15, g_shift_limit=15,b_shift_limit=15, p=1),
    ToTensorV2()
    ]

train_transform = A.Compose(train_transform)

In [39]:
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index].squeeze()
        image_path = row['images']
        mask_path = row['masks']
        
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, 0) // 255
        
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
        
        return image, mask

In [40]:
train_ds = SegmentationDataset(df, train_transform)

In [41]:
train_ds[0][0].shape

torch.Size([3, 360, 640])

In [42]:
@interact(index=(0, len(df)-1))
def show_images(index=0):
    plt.figure(figsize=(20,10))
    plt.axis=("off")
    plt.subplot(121)
    plt.imshow(train_ds[index][0].cpu().detach().numpy().transpose(1,2,0))
    plt.subplot(122)
    plt.imshow(np.squeeze(train_ds[index][1]).cpu().detach().numpy(), cmap='gray')

interactive(children=(IntSlider(value=0, description='index', max=1839), Output()), _dom_classes=('widget-inte…